In [ ]:
import pandas as pd

dataset = pd.read_csv('ABSA16_Restaurants_Train_English_SB2.csv', sep=',', header= None)
dataset1 = pd.read_csv('ABSA16_Restaurants_Train_SB1_v2.csv', sep=',', header= None)
dataset.append(dataset1)
dataset.head()
header_row = 0
dataset.columns = dataset.iloc[header_row]
dataset = dataset.iloc[1:]
dataset = dataset.rename(columns={'rid': 'id','sentences__sentence__id' :'sid','sentences__sentence__text':'review','Opinions__Opinion__category':'category','Opinions__Opinion__polarity':'polarity'})

dataset['review'] = [str(i) for i in dataset['review']]
dataset['category'] = [str(i) for i in dataset['category']]
dataset['polarity'] = [str(i) for i in dataset['polarity']]

import spacy
nlp = spacy.load('en')
dataset.head()
len(dataset)
dataset.iloc[0:5,1:]

dataset.category.unique()
indexNames = dataset[ dataset['category'] == 'nan' ].index
dataset.drop(indexNames , inplace=True)


In [ ]:
aspect_terms = []
for review in nlp.pipe(dataset.review):
    chunks = [(chunk.root.text) for chunk in review.noun_chunks if chunk.root.pos_ == 'NOUN']
    aspect_terms.append(' '.join(chunks))
dataset['aspect_terms'] = aspect_terms
dataset.head(10)

In [ ]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation

aspect_categories_model = Sequential()
aspect_categories_model.add(Dense(512, input_shape=(6000,), activation='relu'))
aspect_categories_model.add(Dense(12, activation='softmax'))
aspect_categories_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Using TensorFlow backend.


In [ ]:
from keras.preprocessing.text import Tokenizer

vocab_size = 6000 
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(dataset['review'])
aspect_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(dataset['aspect_terms']))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

label_encoder = LabelEncoder()
integer_category = label_encoder.fit_transform(dataset['category'])
dummy_category = to_categorical(integer_category)


In [ ]:
aspect_categories_model.fit(aspect_tokenized, dummy_category, epochs=5, verbose=1)

Epoch 1/5
1435/1435 [==============================] - 2s 2ms/step - loss: 2.3244 - accuracy: 0.2516
Epoch 2/5
1435/1435 [==============================] - 2s 1ms/step - loss: 2.0171 - accuracy: 0.3422
Epoch 3/5
1435/1435 [==============================] - 2s 1ms/step - loss: 1.8554 - accuracy: 0.4146
Epoch 4/5
1435/1435 [==============================] - 2s 1ms/step - loss: 1.6815 - accuracy: 0.4725
Epoch 5/5
1435/1435 [==============================] - 2s 1ms/step - loss: 1.5117 - accuracy: 0.5303


In [ ]:
with open('outfile.txt') as out:
  bag = out.read().splitlines()

for i in bag:
  chunks = [(chunk.root.text) for chunk in nlp(i).noun_chunks if chunk.root.pos_ == 'NOUN']
  new_review_aspect_terms = ' '.join(chunks)
  new_review_aspect_tokenized = tokenizer.texts_to_matrix([new_review_aspect_terms])
  new_review_category = label_encoder.inverse_transform(aspect_categories_model.predict_classes(new_review_aspect_tokenized))
  print(new_review_category)

In [ ]:
sentiment_terms = []
for review in nlp.pipe(dataset['review']):
        if review.is_parsed:
            sentiment_terms.append(' '.join([token.lemma_ for token in review if (not token.is_stop and not token.is_punct and (token.pos_ == "ADJ" or token.pos_ == "VERB"))]))
        else:
            sentiment_terms.append('')  
dataset['sentiment_terms'] = sentiment_terms
dataset.head(10)
dataset.polarity.unique()
indexNames = dataset[ dataset['polarity'] == 'conflict' ].index
dataset.drop(indexNames , inplace=True)


In [ ]:
sentiment_model = Sequential()
sentiment_model.add(Dense(512, input_shape=(6000,), activation='relu'))
sentiment_model.add(Dense(3, activation='softmax'))
sentiment_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
sentiment_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(dataset.sentiment_terms))


In [ ]:
label_encoder_2 = LabelEncoder()
integer_sentiment = label_encoder_2.fit_transform(dataset.polarity)
dummy_sentiment = to_categorical(integer_sentiment)


In [ ]:
sentiment_model.fit(sentiment_tokenized, dummy_sentiment, epochs=5, verbose=1)

Epoch 1/5
1394/1394 [==============================] - 2s 1ms/step - loss: 0.8573 - accuracy: 0.7109
Epoch 2/5
1394/1394 [==============================] - 2s 1ms/step - loss: 0.6258 - accuracy: 0.7310
Epoch 3/5
1394/1394 [==============================] - 2s 1ms/step - loss: 0.5347 - accuracy: 0.7884
Epoch 4/5
1394/1394 [==============================] - 2s 1ms/step - loss: 0.4610 - accuracy: 0.8329
Epoch 5/5
1394/1394 [==============================] - 2s 1ms/step - loss: 0.4092 - accuracy: 0.8529


In [ ]:
with open('outfile.txt') as out:
  bag = out.read().splitlines()

for i in bag:
  chunks = [(chunk.root.text) for chunk in nlp(i).noun_chunks if chunk.root.pos_ == 'NOUN']
  new_review_aspect_terms = ' '.join(chunks)
  new_review_aspect_tokenized = tokenizer.texts_to_matrix([new_review_aspect_terms])
  new_review_category = label_encoder_2.inverse_transform(sentiment_model.predict_classes(new_review_aspect_tokenized))
  print(new_review_category)

In [ ]:
with open('outfile.txt') as out:
  test_reviews = out.read().splitlines()

# Aspect preprocessing
test_reviews = [review.lower() for review in test_reviews]
test_aspect_terms = []
for review in nlp.pipe(test_reviews):
    chunks = [(chunk.root.text) for chunk in review.noun_chunks if chunk.root.pos_ == 'NOUN']
    test_aspect_terms.append(' '.join(chunks))
test_aspect_terms = pd.DataFrame(tokenizer.texts_to_matrix(test_aspect_terms))
                             
# Sentiment preprocessing
test_sentiment_terms = []
for review in nlp.pipe(test_reviews):
        if review.is_parsed:
            test_sentiment_terms.append(' '.join([token.lemma_ for token in review if (not token.is_stop and not token.is_punct and (token.pos_ == "ADJ" or token.pos_ == "VERB"))]))
        else:
            test_sentiment_terms.append('') 
test_sentiment_terms = pd.DataFrame(tokenizer.texts_to_matrix(test_sentiment_terms))

# Models output
test_aspect_categories = label_encoder.inverse_transform(aspect_categories_model.predict_classes(test_aspect_terms))
test_sentiment = label_encoder_2.inverse_transform(sentiment_model.predict_classes(test_sentiment_terms))
for i in range(61,70):
    print(dataset.iloc[i,2] + " is expressing a  " + test_sentiment[i] + " opinion about " + test_aspect_categories[i])